# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
weather_df = pd.read_csv('../WeatherPy/output_data/cities.csv')
weather_df.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mar del Plata,-38.0023,-57.5575,64.99,67,0,5.75,AR,1615600548
1,Atuona,-9.8000,-139.0333,79.79,58,0,0.98,PF,1615600549
2,Pouébo,-20.4000,164.5667,82.18,68,26,12.06,NC,1615600814
3,Rundu,-17.9333,19.7667,65.43,63,63,3.80,NaN,1615600814
4,Khatanga,71.9667,102.5000,-16.35,79,100,12.77,RU,1615600529


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
gmaps.configure(api_key=g_key)
#store coordinates
coordinates = weather_df[['Latitude','Longitude']]
weights = weather_df['Humidity'].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=weights, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
ideal_df = pd.DataFrame(weather_df, columns = ['City','Latitude','Longitude','Max Temperature', 'Wind Speed', 'Cloudiness','Country'])

ideal_temp = (ideal_df["Max Temperature"] < 80) & (ideal_df["Max Temperature"] > 70)
ideal_wind = ideal_df["Wind Speed"] < 10
ideal_clouds = ideal_df["Cloudiness"] == 0

hotel_df = ideal_df[ideal_temp & ideal_wind & ideal_clouds]

hotel_df

,City,Latitude,Longitude,Max Temperature,Wind Speed,Cloudiness,Country
1,Atuona,-9.8000,-139.0333,79.79,0.98,0,PF
12,Cidreira,-30.1811,-50.2056,71.20,5.32,0,BR
45,Qandala,11.4720,49.8728,76.32,6.87,0,SO
135,Hualmay,-11.0964,-77.6139,72.01,7.07,0,PE
155,Rawson,-43.3002,-65.1023,70.14,9.19,0,AR
266,Nioro,13.3500,-15.7500,74.95,7.58,0,GM
348,Talara,-4.5772,-81.2719,78.80,5.75,0,PE
387,Black River,18.0264,-77.8487,77.00,8.05,0,JM
416,Cayenne,4.9333,-52.3333,73.40,7.36,0,GF
445,Caravelas,-17.7125,-39.2481,73.11,5.30,0,BR


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
hotel_df['Hotel Name'] = ""
hotel_df

<ipython-input-44-cb2d5672db59>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Latitude,Longitude,Max Temperature,Wind Speed,Cloudiness,Country,Hotel Name
1,Atuona,-9.8000,-139.0333,79.79,0.98,0,PF,
12,Cidreira,-30.1811,-50.2056,71.20,5.32,0,BR,
45,Qandala,11.4720,49.8728,76.32,6.87,0,SO,
135,Hualmay,-11.0964,-77.6139,72.01,7.07,0,PE,
155,Rawson,-43.3002,-65.1023,70.14,9.19,0,AR,
266,Nioro,13.3500,-15.7500,74.95,7.58,0,GM,
348,Talara,-4.5772,-81.2719,78.80,5.75,0,PE,
387,Black River,18.0264,-77.8487,77.00,8.05,0,JM,
416,Cayenne,4.9333,-52.3333,73.40,7.36,0,GF,
445,Caravelas,-17.7125,-39.2481,73.11,5.30,0,BR,


In [45]:
indices = []
indices = list(hotel_df.index.values)
indices

[1, 12, 45, 135, 155, 266, 348, 387, 416, 445, 446, 469, 514, 520, 556]

In [46]:
hotels = []

for index in indices:
    lat = hotel_df.loc[index]['Latitude']
    lng = hotel_df.loc[index]['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    results = response.json()
    try:
        hotels.append(results['results'][0]['name'])
    except:
        hotels.append("")
        
hotel_df['Hotel Name'] = hotels
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

<ipython-input-46-74cf6eb38ba2>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = hotels


,City,Latitude,Longitude,Max Temperature,Wind Speed,Cloudiness,Country,Hotel Name
1,Atuona,-9.8000,-139.0333,79.79,0.98,0,PF,Atuona
12,Cidreira,-30.1811,-50.2056,71.20,5.32,0,BR,CIDREIRA
45,Qandala,11.4720,49.8728,76.32,6.87,0,SO,Qandala
135,Hualmay,-11.0964,-77.6139,72.01,7.07,0,PE,Huacho
155,Rawson,-43.3002,-65.1023,70.14,9.19,0,AR,Rawson


In [47]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[['Latitude', 'Longitude']]

In [48]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))